### 1. data load

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
#  data load
df2017 = pd.read_csv("./data/2017_사업보고서_01_재무상태표_연결_20200626.txt", sep = "\t", encoding = "cp949")

In [ ]:
df = pd.read_csv("./data/2017_2019_without_entity.csv")

### 2. 연속형변수 변경 dtype 변경

In [2]:
def str_to_float(data):
    import numpy as np
    
    data["당기"] = data["당기"].str.replace(",", "")
    data["전기"] = data["전기"].str.replace(",", "")
    data["전전기"] = data["전전기"].str.replace(",", "")
    data["당기"] = data["당기"].astype(np.float32)
    return data

df2017 = str_to_float(df2017)

### 3. 불필요한 변수 제거

In [3]:
# 필요 없는 변수 제거
def delete_col(data):
    data.drop("통화", axis = 1, inplace = True)
    data.drop("Unnamed: 15", axis = 1, inplace = True)
    data.drop("당기", axis = 1, inplace = True)
    data.drop("전전기", axis = 1, inplace = True)
    data.drop("재무제표종류", axis = 1, inplace = True)
    return data

df2017 = delete_col(df2017)

### 4. 항목명 변수 처리

In [4]:
def duplication_check(data):
    # 항목코드 중복 확인
    entity_코드 = []
    entity_명 = []
    without_entity_코드 = []
    without_entity_명 = []

    # 각 리스트에 코드 및 코드명 append(소문자 처리)
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            entity_코드.append(data["항목코드"][idx].lower())
            entity_명.append(data["항목명"][idx].lower())
        else:
            without_entity_코드.append(data["항목코드"][idx].lower())
            without_entity_명.append(data["항목명"][idx].lower())

    # !entity 코드 딕셔너리
    dic_without_entity = {re.findall("_\w{1,}", without_entity_코드[idx])[0].lower():[] for idx in range(len(without_entity_코드))}

    for i, j in zip(without_entity_코드, without_entity_명):
        dic_without_entity[re.findall("_\w{1,}", i)[0].lower()].append(j)

    check_without_entity = {i:len(set(j)) for i, j in dic_without_entity.items()}
    
    # entity 코드 딕셔너리
    dic_entity = {entity_코드[idx]:[] for idx in range(len(entity_코드))}
    
    for i, j in zip(entity_코드, entity_명):
        dic_entity[i].append(j)
        
    check_entity = {i:len(set(j)) for i, j in dic_entity.items()}
    
    return entity_코드, entity_명, without_entity_코드, without_entity_명, dic_without_entity, dic_entity

entity_코드, entity_명, without_entity_코드, without_entity_명, dic_without_entity, dic_entity = duplication_check(df2017)

### 5. 공시된 항목명, 항목코드 가져오기(금융감독원 재무제표 양식)

In [5]:
def extract_element_id(data):
    element_idx_lst_entity = []
    element_idx_lst_without_entity = []
    
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            element_idx_lst_entity.append(data["항목코드"][idx])
        else:
            element_idx_lst_without_entity.append(data["항목코드"][idx])
            
    entity = sorted(set(element_idx_lst_entity), reverse = True)
    without_entity = sorted(set(element_idx_lst_without_entity), reverse = True)
    
    return entity, without_entity

a, b = extract_element_id(df2017)

elementid = pd.read_excel("./data/재무제표양식.xlsx", encoding = "utf-8", sheet_name = "BS1")
# '한글 Label', 'Element ID'
df_element = elementid[['한글 Label', 'Element ID']]
df_element = df_element[df_element["Element ID"].notnull()]

df_element.columns = df_element.columns.str.replace(" ", "_")
lst_element = df_element["Element_ID"].unique()

re_lst_element = []
for i in lst_element:
    a = re.findall("_\w{1,}", i)[0].lower()
    re_lst_element.append(a)
 
dic_element = {i:[] for i in re_lst_element}
for i, j in zip(re_lst_element, df_element["한글_Label"]):
    dic_element[i].append(j)

### 6. entity 제외 항목명 변수 전처리

In [56]:
lst_idx = []
for idx in range(len(df2017)):
    try:
        if "entity" not in df2017["항목코드"][idx]:
            df2017["항목명"][idx] = dic_element[re.findall("_\w{1,}", df2017["항목코드"][idx])[0].lower()][0]
    except Exception as e:
        lst_idx.append(idx)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [72]:
df2017.iloc[lst_idx, :]["항목명"].unique()

array(['   납입자본', '      기타비유동자산', '      기타유동자산', '      기타유동부채',
       '      기타비유동비금융자산', '   자본금', '      기타비유동부채', '      기타채권',
       '      기타비유동비금융부채', '      기타유동비금융자산', '      기타유동비금융부채',
       '      기타비유동채권', '      기타자본', '      단기매도가능증권', '      비유동채권',
       '      기타지급채무', '      당기손익인식금융자산', '      매도가능금융자산',
       '      비유동기타채권', '      기타금융부채',
       '         매각예정으로 분류된 비유동자산이나 처분자산집단',
       '         소유주에 대한 분배예정으로 분류된 비유동자산이나 처분자산집단', '         기타단기충당부채',
       '      기타유동채무'], dtype=object)

In [75]:
df2017.iloc[lst_idx, :]["항목코드"].unique()

array(['dart_ContributedEquity', 'ifrs_OtherNoncurrentNonfinancialAssets',
       'ifrs_OtherCurrentNonfinancialAssets',
       'ifrs_OtherCurrentNonfinancialLiabilities',
       'ifrs_NoncurrentPayables',
       'ifrs_OtherNoncurrentNonfinancialLiabilities',
       'dart_NonCurrentReceivablesGross', 'ifrs_OtherEquityInterest',
       'ifrs_NoncurrentAssetsOrDisposalGroupsClassifiedAsHeldForSale',
       'ifrs_NoncurrentAssetsOrDisposalGroupsClassifiedAsHeldForDistributionToOwners',
       'ifrs_OtherShorttermProvisions'], dtype=object)

In [74]:
df2017.iloc[lst_idx, :][["회사명", "항목명", "항목코드"]]

,회사명,항목명,항목코드
37,3S,납입자본,dart_ContributedEquity
4820,SBI액시즈,기타비유동자산,ifrs_OtherNoncurrentNonfinancialAssets
4826,SBI액시즈,기타유동자산,ifrs_OtherCurrentNonfinancialAssets
4840,SBI액시즈,기타유동부채,ifrs_OtherCurrentNonfinancialLiabilities
4843,SBI액시즈,납입자본,dart_ContributedEquity
...,...,...,...
72260,형지엘리트,소유주에 대한 분배예정으로 분류된 비유동자산이나 처분자산집단,ifrs_NoncurrentAssetsOrDisposalGroupsClassifie...
72283,형지엘리트,기타단기충당부채,ifrs_OtherShorttermProvisions
73120,효성오앤비,기타유동자산,ifrs_OtherCurrentNonfinancialAssets
73134,효성오앤비,기타유동채무,ifrs_OtherCurrentNonfinancialLiabilities


In [71]:
df2017.to_csv("./data/2017_2017변수확인.csv", encoding = "euc-kr", index = False)

In [34]:
df2017.iloc[37,:] = None
df2017.iloc[58,:] = None
df2017.iloc[59,10] = "ifrs-full_PropertyPlantAndEquipment"
df2017.iloc[69,:] = None
df2017.iloc[70,10] = "ifrs-full_TradeAndOtherCurrentPayables"
df2017.iloc[78,:] = None

In [95]:
df = pd.read_csv("./data/2017_사업보고서_01_재무상태표_연결_20200626.txt", sep = "\t", encoding = "cp949")
df.iloc[37,:]

재무제표종류          재무상태표, 유동/비유동법-연결재무제표
종목코드                         [060310]
회사명                                3S
시장구분                        코스닥시장상장법인
업종                                262
업종명                          전자부품 제조업
결산월                                 3
결산기준일                      2017-03-31
보고서종류                           사업보고서
통화                                KRW
항목코드           dart_ContributedEquity
항목명                              납입자본
당기                     20,987,069,000
전기                     20,446,134,000
전전기                    20,446,134,000
Unnamed: 15                       NaN
Name: 37, dtype: object

In [24]:
df = pd.read_csv("./data/2018_사업보고서_01_재무상태표_연결_20200627.txt", sep = "\t", encoding = "cp949")
df.iloc[56:64,:]

,재무제표종류,종목코드,회사명,시장구분,업종,업종명,결산월,결산기준일,보고서종류,통화,항목코드,항목명,당기,전기,전전기,Unnamed: 15
56,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,763,산업용 기계 및 장비 임대업,12,2018-12-31,사업보고서,KRW,entity00365387_udf_BS_2017111322159142_Noncurr...,매도가능금융자산,NaN,"24,745,500,552","21,855,801,173",NaN
57,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,763,산업용 기계 및 장비 임대업,12,2018-12-31,사업보고서,KRW,ifrs_NoncurrentFinancialAssetsMeasuredAtFairVa...,기타포괄손익-공정가치 측정 금융자산,"7,352,135,211",NaN,NaN,NaN
58,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,763,산업용 기계 및 장비 임대업,12,2018-12-31,사업보고서,KRW,entity00365387_udf_BS_2019322151417802_Noncurr...,당기손익인식공정가치측정금융자산,"23,045,100,547",NaN,NaN,NaN
59,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,763,산업용 기계 및 장비 임대업,12,2018-12-31,사업보고서,KRW,entity00365387_udf_BS_20171113195237304_Noncur...,관계회사투자,"15,112,548,198","29,416,917,460","13,191,603,476",NaN
60,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,763,산업용 기계 및 장비 임대업,12,2018-12-31,사업보고서,KRW,dart_RentalAssetGross,렌탈자산,"589,658,906,950","1,477,994,648,458","1,305,514,644,671",NaN
61,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,763,산업용 기계 및 장비 임대업,12,2018-12-31,사업보고서,KRW,ifrs_PropertyPlantAndEquipment,유형자산,"245,055,468,592","332,064,561,991","252,633,766,155",NaN
62,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,763,산업용 기계 및 장비 임대업,12,2018-12-31,사업보고서,KRW,ifrs_IntangibleAssetsOtherThanGoodwill,무형자산,"79,481,799,795","69,978,715,694","55,569,122,687",NaN
63,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,763,산업용 기계 및 장비 임대업,12,2018-12-31,사업보고서,KRW,ifrs_InvestmentProperty,투자부동산,"1,904,623,279","810,603,649","23,493,318,807",NaN


In [26]:
df = pd.read_csv("./data/2016_사업보고서_01_재무상태표_연결_20200626.txt", sep = "\t", encoding = "cp949")
df.iloc[56:64,:] 

,재무제표종류,종목코드,회사명,시장구분,업종,업종명,결산월,결산기준일,보고서종류,통화,항목코드,항목명,당기,전기,전전기,Unnamed: 15
56,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,693,산업용 기계 및 장비 임대업,12,2016-12-31,사업보고서,KRW,entity00365387_udf_BS_2015330194116968_Noncurr...,매도가능금융자산,"21,855,801,173","8,166,566,599","6,820,303,439",NaN
57,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,693,산업용 기계 및 장비 임대업,12,2016-12-31,사업보고서,KRW,entity00365387_udf_BS_2016328183229104_Noncurr...,관계회사투자,"13,191,603,476","4,308,690,539",NaN,NaN
58,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,693,산업용 기계 및 장비 임대업,12,2016-12-31,사업보고서,KRW,ifrs_PropertyPlantAndEquipment,유형자산,"1,558,148,410,826","1,357,205,726,579","1,174,993,919,428",NaN
59,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,693,산업용 기계 및 장비 임대업,12,2016-12-31,사업보고서,KRW,ifrs_InvestmentProperty,투자부동산,"23,493,318,807","23,704,631,853","28,608,271,334",NaN
60,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,693,산업용 기계 및 장비 임대업,12,2016-12-31,사업보고서,KRW,ifrs_IntangibleAssetsOtherThanGoodwill,무형자산,"55,569,122,687","35,297,529,072","52,541,271,055",NaN
61,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,693,산업용 기계 및 장비 임대업,12,2016-12-31,사업보고서,KRW,ifrs_OtherNoncurrentFinancialAssets,기타비유동금융자산,"48,102,016,035","29,423,879,228","18,150,165,415",NaN
62,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,693,산업용 기계 및 장비 임대업,12,2016-12-31,사업보고서,KRW,ifrs_OtherNoncurrentNonfinancialAssets,기타비유동자산,"8,614,515,819","8,547,639,885","6,541,009,675",NaN
63,"재무상태표, 유동/비유동법-연결재무제표",[095570],AJ네트웍스,유가증권시장상장법인,693,산업용 기계 및 장비 임대업,12,2016-12-31,사업보고서,KRW,ifrs_DeferredTaxAssets,이연법인세자산,"6,450,854,388","5,257,520,707","5,016,091,701",NaN
